# Deep Learning Framework for Predicting Himalayan Summit Success 


In [10]:
from torch import nn
from torchvision import transforms

try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[0]) >= 2, "torch version should be 2.+"
    assert int(torchvision.__version__.split(".")[1]) >= 15, "torchvision version should be 0.15+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not correct. Installing correct versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

try:
    import matplotlib.pyplot as plt
except ImportError:
    print("[INFO] Couldn't find matplotlib...installing it")
    !pip install -q matplotlib
    import matplotlib.pyplot as plt

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from tqdm.auto import tqdm
except:
    print(f"[INFO] Couldnt't find tqdm... installing it ")
    !pip install tqdm
    from tqdm.auto import tqdm

try:
    from torchinfo import summary
except ImportError:
    print("[INFO] Couldn't find torchinfo... installing it")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from dbfread import DBF
except ImportError:
    print("[INFO] Coudln't find dbfread...installing it")
    !pip install -q dbfread
    from dbfread import DBF


try:
    from torch.utils.tensorboard import SummaryWriter
except:
    print("[INFO] Couldn't find tensorboard... installing it.")
    !pip install -q tensorboard
    from torch.utils.tensorboard import SummaryWriter

try:
    import torchmetrics, mlxtend
    print(f"mlextend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >- 19
except:
    !pip install -q torchmetrics -U mlxtend
    import torchmetrics, mlxtend
    print(f"mlextend version: {mlxtend.__version__}")

try:
    import cdsapi
except ImportError:
    print("[INFO] Coudldn't find cdsapi...installing it.")
    !pip install -q cdsapi
    import cdsapi

try:
    import pandas as pd
except ImportError:
    print("[INFO] Couldn't find pandas... installing it")
    !pip install -q pandas
    import pandas as pd

try:
    from einops import rearrange, repeat
except ImportError:
    print("[INFO] Couldn't find einops... installing it")
    !pip install -q einops
    from einops import rearrange, repeat


torch version: 2.6.0
torchvision version: 0.21.0
mlextend version: 0.23.4


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [12]:
# We adjust the PYTHONPATH to keep absolute imports
import sys
sys.path.append("src")

In [13]:
from pathlib import Path

himalayan_train_dir = Path("data/himalayas_data/train")
himalayan_val_dir = Path("data/himalayas_data/val")
himalayan_test_dir = Path("data/himalayas_data/test")

himalayan_train_file = himalayan_train_dir / "train.csv"
himalayan_val_file = himalayan_val_dir / "val.csv"
himalayan_test_file = himalayan_test_dir / "test.csv"

In [14]:
df_train = pd.read_csv(himalayan_train_file)

print(f"First 10 rows:\n{df_train.head(10)}")
print(f"First training instance:\n{df_train.iloc[0]}")
print(f"Instance shape:\n{df_train.iloc[0].shape}")

First 10 rows:
  SEX      CITIZEN      STATUS  MO2USED  MROUTE1  SEASON  O2USED  CALCAGE  \
0   M        Japan     Climber     True        1       3    True       49   
1   M        Spain     Climber    False        1       3   False       54   
2   F  Switzerland     Climber    False        1       3    True       25   
3   M        Nepal  H-A Worker     True        1       1    True       31   
4   M          USA     Climber     True        1       1    True       60   
5   M      Bahrain     Climber    False        1       3    True       34   
6   M        Chile     Climber     True        1       1    True       42   
7   F        Japan      Leader     True        0       4    True       43   
8   M        India     Climber     True        1       1    True       48   
9   M      S Korea     Climber    False        0       3   False       29   

   HEIGHTM  MDEATHS  HDEATHS  SMTMEMBERS  SMTHIRED  Target  
0     8163        0        0          14        11       1  
1     8163     

In [15]:
from src.data_setup import train_dataloader, val_dataloader, test_dataloader

train_dataloader, val_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x311f4af50>,
 <torch.utils.data.dataloader.DataLoader at 0x311ff9e90>)

In [16]:
cat_batch, cont_batch, label_batch, cat_mask_batch, cont_mask_batch = next(iter(train_dataloader))

# First instance
cat_instance = cat_batch[0]
cont_instance = cont_batch[0]
label_instance = label_batch[0]

print(f"Categorical instance: {cat_instance}\nCategorical instance shape: {cat_instance.shape}\n")
print(f"Continuous instance: {cont_instance}\nContinuous instance shape: {cont_instance.shape}\n")
print(f"Label instance: {label_instance}\nLabel instance shape: {label_instance.shape}\n")

Categorical instance: tensor([ 0,  1, 77, 60,  0,  0,  0,  1])
Categorical instance shape: torch.Size([8])

Continuous instance: tensor([-1.2303,  0.9899, -0.2801, -0.1664, -0.8072, -0.6175])
Continuous instance shape: torch.Size([6])

Label instance: 0
Label instance shape: torch.Size([])



In [24]:
# from src.tab_transformer.tab_train import train_step
from src.tab_transformer.tab_model import SAINT
from src.data_setup import continuous_columns, categorical_columns
import numpy as np

# Returns the amount of unique values per categorical column
cat_dims = [len(np.unique(df_train[col])) for col in categorical_columns]

# Hyperparameter selection based on default original architecture instantiation
saint = SAINT(
    categories = tuple(cat_dims), 
    num_continuous = len(continuous_columns),                
    dim = 32,                           
    dim_out = 1,                       
    depth = 6,                       
    heads = 8,                         
    attn_dropout = 0.1,             
    ff_dropout = 0.1,                  
    mlp_hidden_mults = (4, 2),       
    cont_embeddings = 'MLP',
    attentiontype = 'colrow',
    final_mlp_style = 'sep',
    y_dim = 2 # Binary classification
)

[INIT] #categories=7, #unique_categories=603, #special_tokens=0
[INIT] total_tokens (vocab size) = 603
[INIT] #continuous features = 6, cont_embeddings = 'MLP'
[INIT] embedding dim = 32, attentiontype = 'colrow', final_mlp_style = 'sep'
[INIT] computed input_size for final MLP = 416, nfeats = 13
[INIT] Transformer built: RowColTransformer
[INIT] Head MLP dims = [416, 208, 104, 1]
[INIT] Mask & position embeddings created


In [ ]:
# from torchinfo import summary
# import torch

# summary(model=model,
#         # input_size=[(32, 8, 32), (32, 6, 32)],
#         input_size=(416, 13),
#         dtypes=[torch.float, torch.float],
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
# )

In [ ]:
from src.tab_transformer.tab_train import train_step
                              )